






































+# Managing Imports


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision.models import resnet18, ResNet18_Weights
import matplotlib.pyplot as plt
import time
import copy
import os
import cv2
import torch
from PIL import Image
import torchvision.transforms as transforms # not to be confused with tensorflow
from torchvision.transforms.functional import crop
from io import BytesIO
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, random_split

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/My Drive/Machine Learning/COSMOS/Final Project/Emotion Recognition/dset.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = torch.load(path)
train_length = int(0.9 * len(data))
train_data, test_data = torch.utils.data.random_split(data, [train_length, len(data) - train_length])

torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Building the Model and Getting the Dataset


In [ ]:
# using resnet18 from the pytorch library
model = resnet18(num_classes = 6).to(device)
#model.fc.register_forward_hook(lambda m, inp, out: F.dropout(out, p=0.5, training=m.training))

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.0005, momentum = 0.9, weight_decay = 1e-5)

val_acc = []
val_loss = []

trainbsize = 37
testbsize = 5

training_acc = []
trainin_loss = []
train_loader = DataLoader(train_data, batch_size = trainbsize, shuffle = True, pin_memory = True, num_workers = 2)
test_loader = DataLoader(test_data, batch_size = testbsize, shuffle = False, pin_memory = True, num_workers = 2)

In [ ]:
transform = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

More Data Preprocessing

In [ ]:
def preprocess_again(image, label, batch):
  label = label.squeeze(1)
  return image, label

# Training Loop

In [ ]:
epochs = 19

val_acc = []
val_loss = []
training_acc = []
training_loss = []

for epoch in range(1, epochs+1):
  model.train()
  train_loss = 0.00
  correct = 0.00
  total = 0.00
  for image, label in train_loader:
    image, label = preprocess_again(image, label, trainbsize)
    image = image.cuda()
    label = label.cuda()
    optimizer.zero_grad()
    y_predict = model(image).to(device)
    loss = criterion(y_predict, label)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()/len(train_loader)
    _, prediction = y_predict.max(dim=1)
    correct += (prediction.eq(label).sum().item())
    total += label.size(0)
  training_acc.append(correct / total)
  training_loss.append(train_loss/len(train_loader))
  print(f"Epoch: [{epoch}/{epochs}], Training Accuracy: {training_acc[epoch - 1]:.4f}, Training Loss: {training_loss[epoch - 1]:.4f}")

Epoch: [1/19], Training Accuracy: 0.3090, Training Loss: 0.0090
Epoch: [2/19], Training Accuracy: 0.3894, Training Loss: 0.0081
Epoch: [3/19], Training Accuracy: 0.4215, Training Loss: 0.0078
Epoch: [4/19], Training Accuracy: 0.4566, Training Loss: 0.0075
Epoch: [5/19], Training Accuracy: 0.4802, Training Loss: 0.0073
Epoch: [6/19], Training Accuracy: 0.5061, Training Loss: 0.0070
Epoch: [7/19], Training Accuracy: 0.5351, Training Loss: 0.0067
Epoch: [8/19], Training Accuracy: 0.5507, Training Loss: 0.0064
Epoch: [9/19], Training Accuracy: 0.5755, Training Loss: 0.0061
Epoch: [10/19], Training Accuracy: 0.6150, Training Loss: 0.0057
Epoch: [11/19], Training Accuracy: 0.6489, Training Loss: 0.0053
Epoch: [12/19], Training Accuracy: 0.6916, Training Loss: 0.0048
Epoch: [13/19], Training Accuracy: 0.7289, Training Loss: 0.0043
Epoch: [14/19], Training Accuracy: 0.7718, Training Loss: 0.0038
Epoch: [15/19], Training Accuracy: 0.8276, Training Loss: 0.0031
Epoch: [16/19], Training Accuracy:

In [ ]:
model.eval()
correct = 0.00
total = 0.00
_val_loss = 0.00

with torch.no_grad():
  for image, label in test_loader:
    image, label = preprocess_again(image, label, testbsize)
    image = image.cuda()
    label = label.cuda()
    y_pred = model(image).to(device)
    _, prediction = y_pred.max(dim=1)
    correct += prediction.eq(label).sum().item()
    total += label.size(0)

print(f"Validation Accuracy: {correct/total:4f}")

Validation Accuracy: 0.520053
